In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:

import torch


In [2]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [4]:
from sklearn.datasets import fetch_california_housing

# get data
data = fetch_california_housing()
X = data['data']
y = data['target']

print(data['DESCR'])

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [ ]:
cali_df = pd.DataFrame(data.data, columns=data.feature_names)
cali_df.head()

In [ ]:
corr = cali_df.corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

cmap = sns.diverging_palette(220, 10, as_cmap=True)

sns.heatmap(corr, mask=mask, cmap=cmap, square=True, linewidths=.5, center=0, vmax=1);

In [5]:
X.shape,X.dtype

((20640, 8), dtype('float64'))

In [6]:
X = X.astype('float32')
y = y.astype('float32').reshape(-1,1)
y.shape

(20640, 1)

In [7]:
inputs = torch.from_numpy(X)
targets = torch.from_numpy(y)



In [8]:
import torch.nn as nn

## Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [9]:
from torch.utils.data import TensorDataset

In [10]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 8.3252e+00,  4.1000e+01,  6.9841e+00,  1.0238e+00,  3.2200e+02,
           2.5556e+00,  3.7880e+01, -1.2223e+02],
         [ 8.3014e+00,  2.1000e+01,  6.2381e+00,  9.7188e-01,  2.4010e+03,
           2.1098e+00,  3.7860e+01, -1.2222e+02],
         [ 7.2574e+00,  5.2000e+01,  8.2881e+00,  1.0734e+00,  4.9600e+02,
           2.8023e+00,  3.7850e+01, -1.2224e+02]]),
 tensor([[4.5260],
         [3.5850],
         [3.5210]]))

The `TensorDataset` allows us to access a small section of the training data using the array indexing notation (`[0:3]` in the above code). It returns a tuple (or pair), in which the first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [11]:
from torch.utils.data import DataLoader

In [12]:
# Define data loader
batch_size = 1000
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [13]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[   6.0516,   29.0000,    6.9581,  ...,    2.1535,   33.6300,
         -117.9500],
        [   2.8750,   26.0000,    5.1698,  ...,    4.1777,   34.9600,
         -120.4500],
        [   7.2982,    4.0000,    7.1844,  ...,    4.7416,   37.3300,
         -121.8100],
        ...,
        [   4.5260,   21.0000,    6.2857,  ...,    2.6454,   33.8000,
         -118.2900],
        [   2.2925,   44.0000,    4.6693,  ...,    2.1901,   39.7300,
         -121.8200],
        [   4.7083,   28.0000,    5.8000,  ...,    2.6964,   33.8800,
         -118.0000]])
tensor([[5.0000],
        [1.2140],
        [3.4300],
        [1.4570],
        [2.2120],
        [1.0880],
        [1.5000],
        [1.9940],
        [2.3960],
        [1.5400],
        [1.8540],
        [0.8510],
        [0.5910],
        [2.4570],
        [0.7270],
        [1.7600],
        [0.8070],
        [2.3240],
        [2.2410],
        [1.3750],
        [2.2500],
        [1.2780],
        [1.1560],
        [2.3180],
        

In [14]:
# Define model
model = nn.Linear(8, 1) #nn.Linear(no of input features,no of targets)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.2546,  0.1365, -0.0084, -0.0081, -0.2817,  0.2600, -0.0805,  0.2543]],
       requires_grad=True)
Parameter containing:
tensor([-0.1031], requires_grad=True)


PyTorch models also have a helpful .parameters method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [ ]:
# Parameters
list(model.parameters())

`torch.randn` creates a tensor with the given shape, with elements picked randomly from a [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution) with mean 0 and standard deviation 1.

Our *model* is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

![matrix-mult](https://i.imgur.com/WGXLFvA.png)


The number of input features is 8 and the no of target features is 1.Therefore our weight matrix is (8,1).
Since we are using linear regression and our data has eight features, our model is

$$ y(X) = \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_3 + \beta_4 x_4 + \beta_5 x_5 + \beta_6 x_6 + \beta_7 x_7 + \beta_8 x_8 + \beta_0. $$


## Adjust weights and biases using gradient descent

We'll reduce the loss and improve our model using the gradient descent optimization algorithm, which has the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

Let's implement the above step by step.

In [15]:
# Import nn.functional
import torch.nn.functional as F

## Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD stands for `stochastic gradient descent`. It is called `stochastic` because samples are selected in batches (often with random shuffling) instead of as a single group.

In [16]:
# Define loss function
loss_fn = F.mse_loss

In [17]:
preds = model(inputs)
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(289696.7812, grad_fn=<MseLossBackward>)


In [19]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-4)

## Train the model

We are now ready to train the model. We'll follow the exact same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data, instead of processing the entire training data in every iteration. Let's define a utility function `fit` which trains the model for a given number of epochs.

In [ ]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            print(pred)
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            print(loss)
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        #for epoch in range(10):
           # print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
            #break

Some things to note above:

* We use the data loader defined earlier to get batches of data for every iteration.

* Instead of updating parameters (weights and biases) manually, we use `opt.step` to perform the update, and `opt.zero_grad` to reset the gradients to zero.

* We've also added a log statement which prints the loss from the last batch of data for every 10th epoch, to track the progress of training. `loss.item` returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [ ]:
fit(20, model, loss_fn, opt)